In [155]:
from pydantic import BaseModel, validator, Field
from typing import Optional, Union
from datetime import datetime
import json
import rdflib
from rdflib import Graph
from pyfuseki import FusekiUpdate

In [157]:
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX name: <https://bibliokeia.com/authorities/name/> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> """

In [64]:
g = Graph()
g.parse('https://www.loc.gov/standards/mads/rdf/v1/examples/n50048460.ttl', format='ttl')
g.serialize('mads/PersonalName.ttl', format='ttl')

<Graph identifier=N606a7fc4f17b4522a6ce839f69bda628 (<class 'rdflib.graph.Graph'>)>

In [180]:
class Uri(BaseModel):
    value: str
    label: str

class Label(BaseModel):
    value: str
    lang: Optional[str]

class ElementList(BaseModel):
    type: str
    elementValue: Label

class Variant(BaseModel):
    type: str
    variantLabel: str
    elementList: list[ElementList]

In [197]:
class Authority(BaseModel):
    type: str
    recordChangeDate: datetime = Field(default=datetime.now())
    authoritativeLabel: Label
    elementList: list[ElementList]
    isMemberOfMADSCollection: str
    hasBroaderAuthority: Optional[list[Uri]]
    subjectOf: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]



In [182]:
with open('mads/authority.json', encoding='utf-8') as file:
    reader = file.read()
    authority = json.loads(reader)
    file.close()
request = Authority(**authority)
request

FileNotFoundError: [Errno 2] No such file or directory: 'mads/authority.json'

subjects

In [143]:
def MakeElement(elementList):
        eList = list()
        for i in elementList:
                e = f"""[ a madsrdf:{i.type} ;
                madsrdf:elementValue "{i.elementValue.value}"{f'@{i.elementValue.lang}' if i.elementValue.lang else ''} ]"""
                eList.append(e)
        return " ".join(eList)


MakeElement(request.elementList)

'[ a madsrdf:TopicElement ;\n                madsrdf:elementValue "Bubo bubo"@en ]'

In [184]:
def MakeVariant(variants):
    vList = list()
    for i in variants:
        v = f"""[ a madsrdf:{i.type},
                madsrdf:Variant ;
            madsrdf:elementList ( {MakeElement(i.elementList)} ) ;
            madsrdf:variantLabel "{i.variantLabel}" ]"""
        vList.append(v)
    return ", ".join(vList)

MakeVariant(request.hasVariant)

'[ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                madsrdf:elementValue "Oliphant, M. O. W." ] [ a madsrdf:FullNameElement ;\n                madsrdf:elementValue "(Margaret Oliphant Wilson),"@en ] [ a madsrdf:DateNameElement ;\n                madsrdf:elementValue "1828-1897" ] ) ;\n            madsrdf:variantLabel "Oliphant, M. O. W. (Margaret Oliphant Wilson), 1828-1897" ]'

In [151]:
request.isMemberOfMADSCollection

str

In [185]:
id = 'sh85017405'
graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            {f'bf:subjectOf {", ".join([ f"<{i.value}>" for i in request.subjectOf])} ;' if request.subjectOf else ''}
            identifiers:local "{id}" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            ri:recordStatus "novo"^^xsd:string ] ; 
            madsrdf:authoritativeLabel "{request.authoritativeLabel.value}"{f'@{request.authoritativeLabel.lang}' if request.authoritativeLabel.lang else ''} ;
            madsrdf:elementList ( {MakeElement(request.elementList)} ) ;
            {f'madsrdf:hasBroaderAuthority {", ".join([ f"<{i.value}>" for i in request.hasBroaderAuthority])} ;' if request.hasBroaderAuthority else ''}
            {f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .
            
        }} 
        }}"""
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX name: <https://bibliokeia.com/authorities/name/> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>     
    INSERT DATA {
        GRAPH <https://bibliokeia.com/authorities/PersonalName/sh85017405>
        {
        <https://bibliokeia.com/authorities/PersonalName/sh85017405> a madsrdf:Authority, 
            madsrdf:PersonalName ;
            
            identifiers:local "sh85017405" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "2023-04-20 20:45:51.110032"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocab

In [203]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')

response = fuseki_update.run_sparql(graph)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

PersonalName

In [186]:
with open('mads/PersonalName.json', encoding='utf-8') as file:
    reader = file.read()
    authority = json.loads(reader)
    file.close()
request = Authority(**authority)
request.dict()

{'type': 'PersonalName',
 'recordChangeDate': datetime.datetime(2023, 4, 20, 20, 45, 51, 110032),
 'authoritativeLabel': {'value': 'Oliphant, Mrs. (Margaret), 1828-1897',
  'lang': 'en'},
 'elementList': [{'type': 'FullNameElement',
   'elementValue': {'value': 'Oliphant', 'lang': None}},
  {'type': 'TermsOfAddressNameElement',
   'elementValue': {'value': 'Mrs.', 'lang': 'en'}},
  {'type': 'FullNameElement',
   'elementValue': {'value': '(Margaret),', 'lang': 'en'}},
  {'type': 'DateNameElement',
   'elementValue': {'value': '1828-1897', 'lang': None}}],
 'isMemberOfMADSCollection': 'https://bibliokeia.com/authorities/PersonalName/',
 'hasBroaderAuthority': None,
 'subjectOf': None,
 'hasCloseExternalAuthority': None,
 'hasVariant': [{'type': 'PersonalName',
   'variantLabel': 'Oliphant, M. O. W. (Margaret Oliphant Wilson), 1828-1897',
   'elementList': [{'type': 'FullNameElement',
     'elementValue': {'value': 'Oliphant, M. O. W.', 'lang': None}},
    {'type': 'FullNameElement',
   

In [188]:
id = 'n50048460'
graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            identifiers:local "{id}" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            ri:recordStatus "novo"^^xsd:string ] ; 
            madsrdf:authoritativeLabel "{request.authoritativeLabel.value}"{f'@{request.authoritativeLabel.lang}' if request.authoritativeLabel.lang else ''} ;
            madsrdf:elementList ( {MakeElement(request.elementList)} ) ;
            {f'madsrdf:hasBroaderAuthority {", ".join([ f"<{i.value}>" for i in request.hasBroaderAuthority])} ;' if request.hasBroaderAuthority else ''}
            {f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .
            
        }} 
        }}"""
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX name: <https://bibliokeia.com/authorities/name/> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>     
    INSERT DATA {
        GRAPH <https://bibliokeia.com/authorities/PersonalName/n50048460>
        {
        <https://bibliokeia.com/authorities/PersonalName/n50048460> a madsrdf:Authority, 
            madsrdf:PersonalName ;
            identifiers:local "n50048460" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "2023-04-20 20:45:51.110032"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizati

GenreForm

In [201]:
with open('mads/GenreForm.json', encoding='utf-8') as file:
    reader = file.read()
    authority = json.loads(reader)
    file.close()
request = Authority(**authority)
request.dict()

{'type': 'GenreForm',
 'recordChangeDate': datetime.datetime(2023, 4, 20, 20, 59, 47, 910492),
 'authoritativeLabel': {'value': 'Fiction', 'lang': 'en'},
 'elementList': [{'type': 'GenreFormElement',
   'elementValue': {'value': 'Fiction', 'lang': 'en'}}],
 'isMemberOfMADSCollection': 'https://bibliokeia.com/authorities/GenreForm/',
 'hasBroaderAuthority': [{'value': 'http://id.loc.gov/authorities/genreForms/gf2014026415',
   'label': 'Literature'}],
 'subjectOf': None,
 'hasCloseExternalAuthority': [{'value': 'http://id.worldcat.org/fast/1423787',
   'label': 'Literature'}],
 'hasNarrowerAuthority': [{'value': 'http://id.loc.gov/authorities/genreForms/gf2014026217',
   'label': 'Literature'},
  {'value': 'http://id.loc.gov/authorities/genreForms/gf2014026220',
   'label': 'Literature'}],
 'hasVariant': [{'type': 'GenreForm',
   'variantLabel': 'Stories',
   'elementList': [{'type': 'GenreFormElement',
     'elementValue': {'value': 'Stories', 'lang': None}}]}]}

In [202]:
id = 'gf2014026339'
graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            identifiers:local "{id}" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "{request.recordChangeDate}"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            ri:recordStatus "novo"^^xsd:string ] ; 
            madsrdf:authoritativeLabel "{request.authoritativeLabel.value}"{f'@{request.authoritativeLabel.lang}' if request.authoritativeLabel.lang else ''} ;
            madsrdf:elementList ( {MakeElement(request.elementList)} ) ;
            {f'madsrdf:hasBroaderAuthority {", ".join([ f"<{i.value}>" for i in request.hasBroaderAuthority])} ;' if request.hasBroaderAuthority else ''}
            {f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
             {f'madsrdf:hasNarrowerAuthority {", ".join([ f"<{i.value}>" for i in request.hasNarrowerAuthority])} ;' if request.hasNarrowerAuthority else ''}
            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .
            
        }} 
        }}"""
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX name: <https://bibliokeia.com/authorities/name/> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>     
    INSERT DATA {
        GRAPH <https://bibliokeia.com/authorities/GenreForm/gf2014026339>
        {
        <https://bibliokeia.com/authorities/GenreForm/gf2014026339> a madsrdf:Authority, 
            madsrdf:GenreForm ;
            identifiers:local "gf2014026339" ;
            madsrdf:adminMetadata [ a ri:RecordInfo ; 
            ri:recordChangeDate "2023-04-20 20:59:47.910492"^^xsd:dateTime ;
            ri:recordContentSource <http://id.loc.gov/vocabulary/organizati